In [1]:
from MT1D import MT1DProblem, MT1DSurvey, MT1DSrc, ZxyRx, Survey, AppResPhaRx
from SimPEG import Maps, DataMisfit
from scipy.constants import mu_0
import numpy as np

In [5]:
rxloc = np.r_[0.]
srcloc = np.r_[0.]
frequency = np.logspace(-3, 2, 25)
rxType = "AppredPha"

if rxType == "Zxy":
    rx = ZxyRx(rxloc, component="both", frequency=frequency)
    rxList = [rx]
elif rxType == "AppredPha":
    rx = AppResPhaRx(rxloc, component="both", frequency=frequency)
    rxList = [rx]
    
src = MT1DSrc(rxList, loc=srcloc)
survey = MT1DSurvey([src])
mesh = survey.setMesh(sigma=0.01, max_depth_core=5000., 
                      ncell_per_skind=10, n_skind=2, 
                      core_meshType="log", max_hz_core=1000.)

sigma = np.ones(mesh.nC) * 0.01

prob = MT1DProblem(mesh, sigmaMap=Maps.ExpMap(mesh), verbose=False)
prob.pair(survey)

>> Smallest cell size = 50 m
>> Padding distance = 316227 m
>> # of padding cells 17
>> # of core cells cells 16


In [6]:
%%time
f = prob.fields(np.log(sigma))
pred = survey.dpred(np.log(sigma), f=f)

CPU times: user 64.7 ms, sys: 3.44 ms, total: 68.2 ms
Wall time: 66.5 ms


In [7]:
pred.reshape((survey.nFreq, 2))

array([[ 102.26977514,   43.73205394],
       [  99.80405995,   44.73788652],
       [ 100.61504688,   45.04174095],
       [ 100.83701264,   44.99056815],
       [ 100.79308747,   44.98012389],
       [ 100.77427024,   44.97575198],
       [ 100.75048553,   44.96936355],
       [ 100.72005731,   44.96141955],
       [ 100.68117651,   44.95159917],
       [ 100.63147215,   44.9396113 ],
       [ 100.56802477,   44.9252466 ],
       [ 100.48749164,   44.90848916],
       [ 100.38663785,   44.88964977],
       [ 100.26358547,   44.86940965],
       [ 100.11981313,   44.84848684],
       [  99.96162638,   44.82648982],
       [  99.79724425,   44.79993401],
       [  99.62521166,   44.76117591],
       [  99.42075778,   44.70188477],
       [  99.14518993,   44.61997357],
       [  98.77888245,   44.51789991],
       [  98.31583646,   44.39194113],
       [  97.73169511,   44.23445385],
       [  96.99318024,   44.03884103],
       [  96.0608984 ,   43.797756  ]])

In [30]:
from SimPEG import Tests
def derChk(m):
    return [survey.dpred(m), lambda mx: prob.Jvec(m, mx)]
Tests.checkDerivative(derChk, np.log(sigma), plotIt=False, num=3, eps=1e-20, dx=np.log(sigma)*2)

==================== checkDerivative ====================
iter    h         |ft-f0|   |ft-f0-h*J0*dx|  Order
---------------------------------------------------------
 0   1.00e-01    7.801e+02     3.184e+02      nan
 1   1.00e-02    4.868e+01     2.300e+00      2.141
 2   1.00e-03    4.662e+00     3.300e-02      1.843
========================= PASS! =========================
Go Test Go!



True

In [31]:
survey.dobs = pred
dmis = DataMisfit.l2_DataMisfit(survey)
m0 = np.log(sigma)*2.

SimPEG.DataMisfit.l2_DataMisfit assigning default eps of 1e-5 * ||dobs||


In [32]:
Tests.checkDerivative(
    lambda m: [dmis(m), dmis.deriv(m)],
    m0,
    plotIt=False,
    num=4,
    dx = m0*1.
)

==================== checkDerivative ====================
iter    h         |ft-f0|   |ft-f0-h*J0*dx|  Order
---------------------------------------------------------
 0   1.00e-01    2.152e+08     1.364e+08      nan
 1   1.00e-02    8.617e+06     7.373e+05      2.267
 2   1.00e-03    7.950e+05     6.993e+03      2.023
 3   1.00e-04    7.887e+04     6.988e+01      2.000
========================= PASS! =========================
That was easy!



True

In [24]:
v = np.random.rand(mesh.nC)
w = np.random.rand(pred.shape[0])
wtJv = w.dot(prob.Jvec(m0, v))
vtJtw = v.dot(prob.Jtvec(m0, w))
passed = np.abs(wtJv - vtJtw) < 1e-10
print('Adjoint Test', np.abs(wtJv - vtJtw), passed)

('Adjoint Test', 2.1827872842550278e-11, True)
